<a href="https://colab.research.google.com/github/Gustavo-dev04/Analise-de-dados-de-acoes-com-ia/blob/main/Modelo_IA_Analise_acoes_nvidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 Treinamento e Previsão de Dados Financeiros via Regressão Linear com PLN Avançado  

**Autor:** Gustavo Barros  
**Data:** 06/04/2025  
**Versão:** 1.0  

## **📌 Descrição**  
Este projeto utiliza **Processamento de Linguagem Natural (PLN)** avançado e modelos de **regressão linear** para prever tendências financeiras (preços de ativos, indicadores econômicos) a partir de:  
- Dados estruturados (séries temporais financeiras).  
- Dados não estruturados (notícias, relatórios, redes sociais).  

**Objetivos:**  
1. Coletar e processar dados multimodais (numéricos + textuais).  
2. Treinar um modelo preditivo híbrido (PLN + regressão).  
3. Automatizar insights para apoio a decisões.  

## **⚙️ Métodos e Ferramentas**  
- **Bibliotecas:** `scikit-learn` (regressão), `spaCy`/`NLTK` (PLN), `yfinance`/`Alpha Vantage` (dados financeiros).  
- **IA Generativa:** Uso de *agents* (Deep Seek, Gemini) para otimização de hiperparâmetros e documentação.  

## **⚠️ Observações**  
- Os dados podem conter ruído (alta volatilidade financeira).  
- O projeto foi acelerado com auxílio de ferramentas de IA (GitHub Copilot, Deep Seek).

## **🔝 Créditos de Cursos Utilizados**
1. Cursos de IA:
- Machine Learning (Deeplearning.ai/Stanford Online)
- AI Engineering (IBM)

2. Curso de PNL:
- Processamento Neural de Linguagem Natural em Português I (Universidade de São Paulo)

3. Cursos de Análise de Dados:
- Data Analytics (Google)
- Advanced Data Analytics (Google)
- Data Analytics Cloud (Google Cloud)

4. Curso de Matemática:
- Mathematics for Machine Learning (Imperial College London)

#**📚 Bibliotecas**
As bibliotecas utilizadas para o processo:

```
# Downloads na interface do google colab
# !python -m textblob.download_corpora  # Dados para análise de sentimentos do TextBlob
# !python -m spacy download en_core_web_sm  # Modelo de NLP em inglês para Spacy
# !python -m nltk.downloader vader_lexicon  # Lexicon para análise de sentimentos VADER
# !pip install yfinance feedparser textblob nltk  # Instalação das bibliotecas principais

# Bibliotecas básicas para manipulação de dados e visualização
import numpy as np  # Operações matemáticas eficientes
import pandas as pd  # Manipulação de dados em DataFrames
import matplotlib.pyplot as plt  # Visualização de dados
import warnings  # Controle de avisos do sistema
warnings.filterwarnings('ignore')  # Ignorar avisos não críticos
import yfinance as yf  # Yahoo Finance API - coleta dados históricos de ações em tempo real

# Machine Learning e Pré-processamento
from sklearn.preprocessing import RobustScaler  # Normalização robusta a outliers
from sklearn.metrics import (  # Métricas de avaliação de modelos
    mean_absolute_error, #  Erro Absoluto Medio
    mean_squared_error,  # Erro Quadratico Medio
    r2_score             # Coeficiente de determinação
)

# Web Scraping e Coleta de Notícias
import requests  # Requisições HTTP para coletar dados da web
from bs4 import BeautifulSoup  # Parseamento de HTML/XML
import feedparser  # Leitura de feeds RSS/Atom (notícias financeiras)
import re  # Expressões regulares para limpeza de texto

# Processamento de Linguagem Natural (NLP)
from textblob import TextBlob  # Análise de sentimentos básica
from textblob.sentiments import NaiveBayesAnalyzer  # Classificador bayesiano para sentimentos
import nltk  # Natural Language Toolkit
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # Análise de sentimentos específica para finanças

# Controle de tempo
import time  # Medição de tempo de execução/pausas
from datetime import datetime, timedelta  # Manipulação de datas

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18, 9)

```






# PARÂMETROS GLOBAIS DE CONFIGURAÇÃO




```
SYMBOL = 'NVDA'  # Ticker da ação a ser analisada (NVIDIA)
DAYS_TO_FETCH = 365 * 5  # Período histórico (5 anos)
TEST_SIZE = 0.2  # Proporção dos dados para teste (20%)
LEARNING_RATE = 0.05  # Taxa de aprendizado para otimização do modelo
EPOCHS = 10000  # Número máximo de iterações de treinamento
VOLATILITY_WINDOW = 20  # Janela para cálculo de volatilidade (20 dias)
REQUEST_TIMEOUT = 15  # Timeout para requisições web (segundos)
MIN_NEWS_ARTICLES = 5  # Mínimo de notícias para análise válida
MAX_NEWS_ARTICLES = 10  # Máximo de notícias a serem processadas
```


# CLASSE DE ANÁLISE DE NOTÍCIAS



```
class GoogleNewsAnalyzer:
    def __init__(self):
        """Inicializa o analisador de notícias com configurações padrão"""
        self.session = requests.Session()  # Session para conexões HTTP persistentes
        self.session.timeout = REQUEST_TIMEOUT  # Configura timeout
        self.vader = SentimentIntensityAnalyzer()  # Analisador de sentimentos VADER

        # Configuração da URL do RSS do Google News para o símbolo
        self.rss_url = f"https://news.google.com/rss/search?q={SYMBOL}+stock&hl=en-US&gl=US&ceid=US:en"

        # Dicionário de termos financeiros para análise complementar
        self.finance_terms = {
            'positive': ['bullish', 'upgrade', 'beat', 'growth', 'raise', 'buy', 'outperform'],
            'negative': ['bearish', 'downgrade', 'miss', 'cut', 'sell', 'underperform']
        }

    def fetch_google_news(self):
        """Coleta notícias do Google News RSS"""
        articles = []
        try:
            # Faz a requisição e parseia o feed RSS
            response = self.session.get(self.rss_url, timeout=REQUEST_TIMEOUT)
            feed = feedparser.parse(response.text)

            # Processa cada entrada no feed
            for entry in feed.entries[:MAX_NEWS_ARTICLES]:
                # Tenta obter a data de publicação (usa data atual como fallback)
                pub_date = datetime.now()
                if hasattr(entry, 'published_parsed'):
                    pub_date = datetime(*entry.published_parsed[:6])

                # Armazena informações relevantes da notícia
                articles.append({
                    'title': entry.title,  # Título da notícia
                    'source': entry.source.title if hasattr(entry, 'source') else 'Google News',  # Fonte
                    'date': pub_date,  # Data de publicação
                    'url': entry.link  # URL original
                })

        except Exception as e:
            print(f"⚠️ Erro ao acessar Google News: {str(e)}")

        return articles

    def _get_fallback_news(self):
        """Gera notícias fictícias como fallback quando há poucas notícias reais"""
        trends = ['high', 'growth', 'drop', 'volatile', 'steady']  # Termos de tendência
        return [{
            'title': f"{SYMBOL} shows {np.random.choice(trends)} amid market {'gains' if np.random.random() > 0.5 else 'losses'}",
            'source': 'Market Trend',
            'date': datetime.now(),
            'url': ''
        } for _ in range(MIN_NEWS_ARTICLES)]

    def analyze_with_vader(self, text):
        """Analisa sentimento usando VADER (especializado para finanças)"""
        return self.vader.polarity_scores(text)['compound']  # Retorna score composto (-1 a 1)

    def analyze_with_textblob(self, text):
        """Analisa sentimento usando TextBlob com NaiveBayes"""
        analysis = TextBlob(text, analyzer=NaiveBayesAnalyzer())
        return analysis.sentiment.p_pos - analysis.sentiment.p_neg  # Diferença de probabilidades

    def analyze_finance_terms(self, text):
        """Analisa termos financeiros específicos no texto"""
        score = 0
        text_lower = text.lower()
        # Pontua termos positivos (+0.2 cada)
        for term in self.finance_terms['positive']:
            if term in text_lower:
                score += 0.2
        # Penaliza termos negativos (-0.3 cada)
        for term in self.finance_terms['negative']:
            if term in text_lower:
                score -= 0.3
        return score

    def analyze_articles(self, articles):
        """Executa análise completa do conjunto de notícias"""
        # Complementa com notícias fictícias se necessário
        if len(articles) < MIN_NEWS_ARTICLES:
            print(f"⚠️ Poucas notícias ({len(articles)}), complementando com análise de tendência...")
            articles.extend(self._get_fallback_news())

        sentiment_results = []
        print("\n📰 Análise Avançada de Notícias (Google News):")

        # Processa cada artigo
        for article in articles[:MAX_NEWS_ARTICLES]:
            text = article['title']

            # Combina múltiplos métodos de análise
            vader_score = self.analyze_with_vader(text)  # 50% peso
            blob_score = self.analyze_with_textblob(text)  # 30% peso
            finance_score = self.analyze_finance_terms(text)  # 20% peso

            # Score combinado ponderado
            combined_score = (vader_score * 0.5 + blob_score * 0.3 + finance_score * 0.2)

            # Classificação por faixas de score
            if combined_score > 0.2:
                emoji = "🚀"
                sentiment = "FORTEMENTE POSITIVO"
            elif combined_score > 0.05:
                emoji = "📈"
                sentiment = "POSITIVO"
            elif combined_score < -0.2:
                emoji = "💥"
                sentiment = "FORTEMENTE NEGATIVO"
            elif combined_score < -0.05:
                emoji = "📉"
                sentiment = "NEGATIVO"
            else:
                emoji = "➖"
                sentiment = "NEUTRO"

            # Exibe resultado da análise
            print(f"{emoji} [{sentiment}] {article['title'][:60]}... ({article['source']})")
            sentiment_results.append(combined_score)

        # Calcula média de sentimentos
        avg_sentiment = np.mean(sentiment_results) if sentiment_results else 0
        print(f"\n🔍 Sentimento Médio: {avg_sentiment:.2f} (Escala: -1 a 1)")

        # Análise de tendência temporal (se houver dados suficientes)
        trend = 0
        if len(articles) > 3 and all('date' in a for a in articles):
            sorted_articles = sorted(articles, key=lambda x: x['date'])
            early_scores = [self.analyze_with_vader(a['title']) for a in sorted_articles[:3]]
            late_scores = [self.analyze_with_vader(a['title']) for a in sorted_articles[-3:]]
            trend = np.mean(late_scores) - np.mean(early_scores)
            print(f"📅 Tendência Temporal: {'Melhorando' if trend > 0 else 'Piorando' if trend < 0 else 'Estável'}")

        return {
            'sentiment': avg_sentiment,  # Sentimento médio
            'trend': trend,  # Direção da tendência
            'recent_impact': avg_sentiment * 1.5  # Impacto ponderado
        }

```



# CLASSE PRINCIPAL DE PREVISÃO


```
class EnhancedStockPredictor:
    def __init__(self):
        """Inicializa o preditor com configurações e modelos"""
        self.learning_rate = LEARNING_RATE  # Taxa de aprendizado
        self.epochs = EPOCHS  # Máximo de épocas
        self.weights = None  # Pesos do modelo (serão aprendidos)
        self.bias = None  # Viés do modelo (será aprendido)
        self.scaler = RobustScaler()  # Normalizador robusto a outliers
        self.news_analyzer = GoogleNewsAnalyzer()  # Analisador de notícias
        # Colunas de features usadas no modelo
        self.feature_cols = ['close', 'volume', 'ma_5', 'ma_20', 'volatility', 'momentum', 'news_sentiment', 'news_trend']

    def prepare_data(self, df):
        """Prepara os dados para treinamento"""
        try:
            # Feature Engineering - cria novas variáveis explicativas
            df['returns'] = df['close'].pct_change()  # Retornos diários
            df['ma_5'] = df['close'].rolling(5).mean()  # Média móvel 5 dias
            df['ma_20'] = df['close'].rolling(20).mean()  # Média móvel 20 dias
            df['volatility'] = df['returns'].rolling(VOLATILITY_WINDOW).std()  # Volatilidade
            df['momentum'] = df['close'] - df['close'].shift(5)  # Momentum 5 dias

            # Adiciona dados de sentimento de notícias
            articles = self.news_analyzer.fetch_google_news()
            news_impact = self.news_analyzer.analyze_articles(articles)
            df['news_sentiment'] = news_impact['sentiment']  # Sentimento médio
            df['news_trend'] = news_impact['trend']  # Tendência temporal

            # Remove valores NA gerados pelos cálculos
            df = df.dropna()

            # Separa features (X) e target (y)
            X = df[self.feature_cols]
            y = df['close'].shift(-1).dropna()  # Preço do próximo dia
            X = X.iloc[:-1]  # Ajusta para coincidir com y

            # Normaliza as features
            X_scaled = self.scaler.fit_transform(X)

            # Divide em conjuntos de treino e teste
            split_idx = int(len(X) * (1 - TEST_SIZE))
            X_train, X_test = X_scaled[:split_idx], X_scaled[split_idx:]
            y_train, y_test = y[:split_idx], y[split_idx:]

            return X_train, X_test, y_train, y_test, df

        except Exception as e:
            print(f"Erro na preparação: {str(e)}")
            # Fallback: usa apenas features técnicas se houver erro com notícias
            self.feature_cols = ['close', 'volume', 'ma_5', 'ma_20', 'volatility', 'momentum']
            return self.prepare_data(df)

    def train(self, X, y):
        """Treina o modelo de regressão linear com gradiente descendente"""
        n_samples, n_features = X.shape
        # Inicializa pesos com pequenos valores aleatórios
        self.weights = np.random.randn(n_features) * 0.01
        self.bias = 0
        best_error = float('inf')  # Para early stopping

        for epoch in range(self.epochs):
            # Predições atuais
            y_pred = np.dot(X, self.weights) + self.bias
            # Cálculo do erro com regularização L2
            error = np.mean((y_pred - y)**2) + 0.001 * np.sum(self.weights**2)

            # Early stopping se não houver melhoria significativa
            if epoch > 100 and error > best_error * 0.999:
                print(f"✅ Convergência na época {epoch}")
                break

            if error < best_error:
                best_error = error

            # Cálculo dos gradientes
            dw = (1/n_samples) * np.dot(X.T, (y_pred - y)) + (0.002 * self.weights)
            db = (1/n_samples) * np.sum(y_pred - y)          

            # Atualização dos parâmetros
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            # Log periódico
            if epoch % 100 == 0:
                print(f"Época {epoch}: MSE = {error:.4f}")

        return best_error

    def predict(self, X):
        """Faz previsões usando o modelo treinado"""
        return np.dot(X, self.weights) + self.bias

    def evaluate(self, X_test, y_test):
        """Avalia o modelo com métricas e gráficos"""
        y_pred = self.predict(X_test)

        # Cálculo das métricas
        mae = mean_absolute_error(y_test, y_pred)  # Erro absoluto médio
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Raiz do erro quadrático
        r2 = r2_score(y_test, y_pred)  # Coeficiente de determinação

        # Plot dos resultados
        plt.figure(figsize=(18, 9))
        plt.plot(y_test.values, label='Real', linewidth=2)
        plt.plot(y_pred, label='Previsto', linestyle='--')
        plt.title(f'Desempenho do Modelo (MAE: ${mae:.2f}, R²: {r2:.2%})')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.show()

        return {'mae': mae, 'rmse': rmse, 'r2': r2}

    def forecast(self, df, days=5):
        """Gera previsões para os próximos dias"""
        try:
            forecasts = []
            last_row = df.iloc[-1][self.feature_cols].values.reshape(1, -1)

            for _ in range(days):
                # Prepara dados para previsão
                X_scaled = self.scaler.transform(last_row)
                pred = self.predict(X_scaled)[0]
                forecasts.append(pred)

                # Atualiza features para próxima previsão
                new_row = last_row.copy()
                new_row[0][0] = pred  # Atualiza preço
                # Atualiza médias móveis
                new_row[0][2] = (new_row[0][0] + last_row[0][0] * 4) / 5  # MA5
                new_row[0][3] = (new_row[0][0] + last_row[0][0] * 19) / 20  # MA20
                last_row = new_row

            # Prepara resultado
            dates = [df.index[-1] + timedelta(days=i+1) for i in range(days)]
            forecast_df = pd.DataFrame({'date': dates, 'forecast': forecasts})
            forecast_df.set_index('date', inplace=True)

            # Plot das previsões
            plt.figure(figsize=(18, 9))
            plt.plot(df['close'].iloc[-30:], label='Histórico', marker='o')
            plt.plot(forecast_df['forecast'], label='Previsão', color='red', linestyle='--', marker='o')
            plt.title(f'Previsão para {SYMBOL} - Próximos {days} dias')
            plt.legend()
            plt.grid(alpha=0.3)
            plt.show()

            return forecast_df

        except Exception as e:
            print(f"Erro na previsão: {str(e)}")
            return None

```

# FUNÇÃO PRINCIPAL



```
def main():
    """Fluxo principal de execução"""
    print(f"\n{'='*50}")
    print(f"=== ANÁLISE DE {SYMBOL} COM NLP NO GOOGLE NEWS ===")
    print(f"{'='*50}\n")

    try:
        # 1. Baixa dados históricos do Yahoo Finance
        print("📊 Baixando dados históricos...")
        start_time = time.time()
        df = yf.download(SYMBOL, period=f"{DAYS_TO_FETCH//365}y", progress=False)
        df = df[['Close', 'Volume']]  # Mantém apenas preço e volume
        df.columns = ['close', 'volume']  # Renomeia colunas
        print(f"✅ Dados baixados - {len(df)} dias | Último preço: ${df['close'].iloc[-1]:.2f}")

        # 2. Prepara dados com análise de notícias
        print("\n🧠 Preparando dados com análise de notícias...")
        predictor = EnhancedStockPredictor()
        X_train, X_test, y_train, y_test, df_processed = predictor.prepare_data(df)
        print(f"✅ Dados preparados - {len(X_train)} treino, {len(X_test)} teste")

        # 3. Treina modelo
        print("\n🔮 Treinando modelo...")
        train_start = time.time()
        final_error = predictor.train(X_train, y_train)
        print(f"✅ Treino concluído em {time.time()-train_start:.1f}s (MSE final: {final_error:.4f})")

        # 4. Avaliação do modelo
        print("\n📊 Avaliando modelo...")
        metrics = predictor.evaluate(X_test, y_test)
        print(f"\n🔍 Resultados:")
        print(f"- MAE: ${metrics['mae']:.2f}")  # Erro médio em dólares
        print(f"- RMSE: ${metrics['rmse']:.2f}")  # Erro quadrático em dólares
        print(f"- R²: {metrics['r2']:.2%}")  # Porcentagem de variância explicada

        # 5. Gera previsões
        print("\n🔮 Gerando previsões...")
        forecast = predictor.forecast(df_processed)
        if forecast is not None:
            print("\n📈 Previsões para os próximos dias:")
            print(forecast.round(2))  # Preços com 2 casas decimais

    except KeyboardInterrupt:
        print("\n⏹️ Execução interrompida pelo usuário")
    except Exception as e:
        print(f"\n❌ Erro: {str(e)}")
    finally:
        print(f"\n⏱️ Tempo total: {time.time()-start_time:.1f} segundos")

if __name__ == "__main__":
    main()
```


